In [3]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier
import os
from tqdm import tqdm
import torch
from joblib import Parallel, delayed, parallel_backend
from statistics import mean
import numpy as np

import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np

/Users/naweenk/miniforge3/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
file_paths = []

dir_list = os.listdir('./Preprocessed')

for i in tqdm(dir_list):
    file_paths.append(f'./Preprocessed/{i}')

In [ ]:
def signal_length(file_path):
    mean_list = []
    for i in os.listdir(file_path):
        if i == "mesh.ply" or i == "Embeddings":
            continue
        path = f"{file_path}/{i}"
        signal, sr = torchaudio.load(path)
        mean_list.append(signal.shape[1])

    return mean(mean_list)


In [ ]:
# signals = Parallel(n_jobs=-1, backend='loky')(delayed(signal_length)(file) for file in tqdm(file_paths))

In [ ]:
# mean_signal_len = mean(signals)

In [ ]:
# mean_signal_len = int(mean_signal_len)

In [ ]:
# mean_signal_len

In [ ]:
def scale_signals(target_len, signal):
    if signal.shape[1] < target_len:
        missing = target_len - signal.shape[1]
        last_dim = (0, missing)
        signal = torch.nn.functional.pad(signal, last_dim)

        return signal

    elif signal.shape[1] > target_len:
        signal = signal[:, :target_len]

        return signal
    
    else:
        return signal

In [6]:
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", run_opts={"device":"mps"})


In [5]:
def get_embeddings(path):
    for i in os.listdir(path):
        if i == "mesh.ply" or i == "Embeddings":
            continue
        signal, fs = torchaudio.load(f"{path}/{i}")
        signal = scale_signals(683433, signal)
        # mfccs = librosa.feature.mfcc(y=signal, sr=fs, n_mfcc=15)
        # delta_mfccs = librosa.feature.delta(mfccs)
        # delta2_mfccs = librosa.feature.delta(mfccs, order=2)
        # embeddings = np.concatenate((mfccs, delta_mfccs, delta2_mfccs)).flatten()
        # print(embeddings.shape)

        embeddings = classifier.encode_batch(signal)
        embeddings = embeddings.cpu().numpy()
        try:
            os.mkdir(f"./{path}/Embeddings")
        except Exception:
            pass
        try:
            np.save(f"./{path}/Embeddings/{i}.npy", embeddings)
        except Exception:
            pass


In [ ]:
# for i in tqdm(file_paths):
#     get_embeddings(i)

In [ ]:
x = Parallel(n_jobs=3, backend='loky')(delayed(get_embeddings)(i) for i in tqdm(file_paths))

In [2]:
from speechbrain.pretrained import EncoderClassifier

/Users/naweenk/miniforge3/envs/demo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torchaudio

In [4]:
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", run_opts={"device":"cpu"})

In [5]:
import speechbrain
speechbrain.__version__

'0.5.16'

In [ ]:
signal shape:  torch.Size([2, 480384])
shape:,  torch.Size([2, 1, 192])

In [11]:
signal, fs = torchaudio.load("2.wav")
print("signal shape: ", signal.shape)
# print("scale")
# signal = scale_signals(683433, signal)
# print("encode")
if signal.shape[0] == 1:
    signal = signal.repeat(2, 1)
print("signal shape: ", signal.shape)
embedding = classifier.encode_batch(signal)
print("shape:, ", embedding.shape)
# print("to numpy")
# embedding = signal
embedding = embedding.cpu().numpy()

signal shape:  torch.Size([1, 480384])
signal shape:  torch.Size([2, 480384])
shape:,  torch.Size([2, 1, 192])
